# Model PyTorch → ONNX

In [1]:
import os
import sys
import subprocess

# ============================================================
# 0. Pastikan onnx & onnxruntime terinstall
# ============================================================
def install_if_missing(package):
    try:
        __import__(package)
    except ImportError:
        print(f"[INFO] {package} belum ada. Menginstall...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

for pkg in ["onnx", "onnxruntime"]:
    install_if_missing(pkg)

In [2]:
# ============================================================
# 1. Import library utama
# ============================================================
import torch
import torch.nn as nn
import torchvision.models as models

In [3]:
# ============================================================
# 2. Definisi Model (sama seperti sebelumnya)
# ============================================================
def get_model(num_classes, variant="large"):
    if variant == "large":
        model = models.mobilenet_v3_large(weights="IMAGENET1K_V1")
        in_feats = model.classifier[0].in_features
    else:
        model = models.mobilenet_v3_small(weights="IMAGENET1K_V1")
        in_feats = model.classifier[0].in_features

    model.classifier = nn.Sequential(
        nn.Linear(in_feats, 128),
        nn.Hardswish(),
        nn.Dropout(p=0.3),
        nn.Linear(128, num_classes)
    )
    return model

In [6]:
# ============================================================
# 3. Fungsi export
# ============================================================
def export_to_onnx(pt_path, onnx_path, num_classes=2, variant="large"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load model
    model = get_model(num_classes=num_classes, variant=variant).to(device)
    state_dict = torch.load(pt_path, map_location=device)
    model.load_state_dict(state_dict)
    model.eval()

    # Dummy input (sesuai training)
    dummy_input = torch.randn(1, 3, 224, 224).to(device)

    # Export
    torch.onnx.export(
        model,
        dummy_input,
        onnx_path,
        input_names=["input"],
        output_names=["output"],
        opset_version=13
    )
    print(f"[INFO] Export sukses: {onnx_path}")

In [9]:
# ============================================================
# 4. Eksekusi untuk dua model
# ============================================================
os.makedirs("models/onnx", exist_ok=True)

export_to_onnx("models/face-expression.pt", "models/onnx/face-expression.onnx")
export_to_onnx("models/pose-recognition.pt", "models/onnx/pose-recognition.onnx")


C:\Users\harry\AppData\Local\Temp\ipykernel_57916\1428721856.py:17: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


[INFO] Export sukses: models/onnx/face-expression.onnx
[INFO] Export sukses: models/onnx/pose-recognition.onnx


# Model ONNX → HEF

In [ ]:
import os
import subprocess

# ============================================================
# 1. Fungsi compile ONNX -> HEF
# ============================================================
def compile_to_hef(onnx_path, hef_path, hw_arch="hailo8"):
    """
    Compile ONNX model to HEF using Hailo Model Compiler.
    hw_arch bisa hailo8 / hailo10h / hailo10h-r depending on device.
    """
    try:
        cmd = [
            "hailo_model_compiler",
            "--onnx", onnx_path,
            "--hw-arch", hw_arch,
            "--output", hef_path
        ]
        print(f"[INFO] Running: {' '.join(cmd)}")
        subprocess.run(cmd, check=True)
        print(f"[INFO] HEF berhasil dibuat: {hef_path}")
    except subprocess.CalledProcessError as e:
        print(f"[ERROR] Gagal compile {onnx_path} -> {hef_path}")
        print(e)

# ============================================================
# 2. Path ONNX dan Output HEF
# ============================================================
onnx_dir = "models/onnx"
hef_dir = "models/hef"
os.makedirs(hef_dir, exist_ok=True)

models = {
    "face-expression": os.path.join(onnx_dir, "/face-expression.onnx"),
    "pose-recognition": os.path.join(onnx_dir, "/pose-recognition.onnx")
}

# ============================================================
# 3. Compile semua model
# ============================================================
for name, onnx_path in models.items():
    hef_path = os.path.join(hef_dir, f"{name}.hef")
    compile_to_hef(onnx_path, hef_path, hw_arch="hailo8")
